# Data Generation Calculation Method

In [ ]:

import csv
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, rdMolDescriptors

# === File paths ===
smiles_csv_path = r'C:\PhD\Project\NIH3T3.csv'
output_csv_path = r'C:\PhD\Prompt_Engineering\Outputs2\DS3_Calc.csv'

# === Feature extraction function ===
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Basic molecular descriptors
    molar_mass = Descriptors.MolWt(mol)
    logp_value = Descriptors.MolLogP(mol)
    num_hb_acceptors = Lipinski.NumHAcceptors(mol)
    num_hb_donors = Lipinski.NumHDonors(mol)
    tpsa_value = rdMolDescriptors.CalcTPSA(mol)
    num_rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    fraction_sp3_carbons = Descriptors.FractionCSP3(mol)
    qed_value = QED.qed(mol)
    molar_refractivity = Descriptors.MolMR(mol)

    # Lipinski rule violations
    lipinski_violations = sum([
        molar_mass > 500,
        logp_value > 5,
        num_hb_acceptors > 10,
        num_hb_donors > 5
    ])

    # Ring info
    ring_info = mol.GetRingInfo()
    atom_rings = ring_info.AtomRings()
    num_rings = len(atom_rings)
    num_aromatic_rings = len([
        r for r in atom_rings if all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in r)
    ])

    # === Toxicophores ===
    toxicophore_smarts = {
        'Amide': '[NX3][CX3](=O)[#6]',
        'Sulfonamide': '[#6][#6]S(=O)(=O)N',
        'Carboxylic_Acid': '[CX3](=O)[OX2H1]',
        'Nitro_Aliphatic': '[#7]N=O',
        'Nitro_Aromatic': 'c1ccc([N+](=O)[O-])cc1',
        'Azide': '[NX3][NX2]=[NX2]',
        'Acyl_Chloride': '[CX3](=O)Cl',
        'Anhydride': '[CX3](=O)OC(=O)[#6]',
        'Hydroxamic_Acid': '[CX3](=O)N[OH]',
        'Beta_Diketone': '[C;!R](=O)[C;!R](=O)',
        'Peroxynitrite': '[O-][N+](=O)O',
        'General_Nitro': '[$([N+](=O)[O-])]'
    }

    toxicophore_hits = 0
    for smarts in toxicophore_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            toxicophore_hits += 1
    total_toxicophores = len(toxicophore_smarts)

    # === Alerting Structural Motifs ===
    alert_smarts = {
        'Aryl_Sulfonamide': '[#6][#6]S(=O)(=O)Nc1ccccc1',
        'Arylamide': 'CCc1ccc(cc1)C(=O)Nc2ccccc2',
        'Acetanilide': '[#6][#6]C(=O)Nc1ccccc1',
        'Primary_Secondary_Amine': '[NX3;H2,H1;!$(NC=O)]',
        'Alkyne': '[#6]=[#6]',
        'Benzoyl_Chloride': 'c1ccccc1C(=O)Cl',
        'Isothiocyanate': 'N=C=S',
        'Nitrile': '[C;H1,H2]#[N]',
        'Azide_Group': 'N=[N+]=[N-]',
        'Isocyanate': 'N=C=O',
        'Nitrobenzene': 'c1ccccc1N(=O)=O',
        'Alkene': 'C=C',
        'Phenol': '[O;H1]-c1ccccc1',
        'Enol': '[C;H2,H1]=[C;H2,H1]-[OH]',
        'Coumarin': 'c1ccc2c(c1)ccc(=O)o2'
    }

    alert_hits = 0
    for smarts in alert_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            alert_hits += 1
    total_alerts = len(alert_smarts)

    # === High Ionization Potential Substructure ===
    high_ip_smarts = '[#6][#6]S(=O)(=O)N'
    high_ip_atoms = mol.GetSubstructMatches(Chem.MolFromSmarts(high_ip_smarts))
    num_high_ip_atoms = len(high_ip_atoms)

    # === Final feature list ===
    return [
        molar_mass,
        logp_value,
        num_hb_acceptors,
        num_hb_donors,
        tpsa_value,
        num_rotatable_bonds,
        num_rings,
        num_aromatic_rings,
        fraction_sp3_carbons,
        f"{toxicophore_hits}/{total_toxicophores}",
        f"{alert_hits}/{total_alerts}",
        qed_value,
        lipinski_violations,
        molar_refractivity,
        num_high_ip_atoms
    ]

# === Read SMILES from input CSV ===
df = pd.read_csv(smiles_csv_path)
smiles_list = df['PUBCHEM_EXT_DATASOURCE_SMILES'].dropna().unique()

# === Write features to output CSV ===
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    header = [
        "SMILES",
        "Molecular_Weight",
        "LogP",
        "H_Acceptors",
        "H_Donors",
        "TPSA",
        "Rotable_Bonds",
        "Rings",
        "Aromatic_Rings",
        "SP3_Carbon",
        "Toxicophore_Presence",
        "Alerting_Structural_Motifs",
        "QED",
        "Lipinskis_Rule",
        "Molar_Refractivity",
        "Atoms_With_High_Ionization_Potential"
    ]
    writer.writerow(header)

    for smiles in smiles_list:
        features = extract_features(smiles)
        if features is None:
            writer.writerow([smiles] + ["INVALID_SMILES"] * (len(header) - 1))
        else:
            writer.writerow([smiles] + features)

print(f"✅ Feature extraction complete. Output saved to: {output_csv_path}")


In [ ]:

import csv
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, rdMolDescriptors

# === File paths ===
smiles_csv_path = r'C:\PhD\Project\PhoP.csv'
output_csv_path = r'C:\PhD\Prompt_Engineering\Outputs2\DS4_Calc.csv'

# === Feature extraction function ===
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Basic molecular descriptors
    molar_mass = Descriptors.MolWt(mol)
    logp_value = Descriptors.MolLogP(mol)
    num_hb_acceptors = Lipinski.NumHAcceptors(mol)
    num_hb_donors = Lipinski.NumHDonors(mol)
    tpsa_value = rdMolDescriptors.CalcTPSA(mol)
    num_rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    fraction_sp3_carbons = Descriptors.FractionCSP3(mol)
    qed_value = QED.qed(mol)
    molar_refractivity = Descriptors.MolMR(mol)

    # Lipinski rule violations
    lipinski_violations = sum([
        molar_mass > 500,
        logp_value > 5,
        num_hb_acceptors > 10,
        num_hb_donors > 5
    ])

    # Ring info
    ring_info = mol.GetRingInfo()
    atom_rings = ring_info.AtomRings()
    num_rings = len(atom_rings)
    num_aromatic_rings = len([
        r for r in atom_rings if all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in r)
    ])

    # === Toxicophores ===
    toxicophore_smarts = {
        'Amide': '[NX3][CX3](=O)[#6]',
        'Sulfonamide': '[#6][#6]S(=O)(=O)N',
        'Carboxylic_Acid': '[CX3](=O)[OX2H1]',
        'Nitro_Aliphatic': '[#7]N=O',
        'Nitro_Aromatic': 'c1ccc([N+](=O)[O-])cc1',
        'Azide': '[NX3][NX2]=[NX2]',
        'Acyl_Chloride': '[CX3](=O)Cl',
        'Anhydride': '[CX3](=O)OC(=O)[#6]',
        'Hydroxamic_Acid': '[CX3](=O)N[OH]',
        'Beta_Diketone': '[C;!R](=O)[C;!R](=O)',
        'Peroxynitrite': '[O-][N+](=O)O',
        'General_Nitro': '[$([N+](=O)[O-])]'
    }

    toxicophore_hits = 0
    for smarts in toxicophore_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            toxicophore_hits += 1
    total_toxicophores = len(toxicophore_smarts)

    # === Alerting Structural Motifs ===
    alert_smarts = {
        'Aryl_Sulfonamide': '[#6][#6]S(=O)(=O)Nc1ccccc1',
        'Arylamide': 'CCc1ccc(cc1)C(=O)Nc2ccccc2',
        'Acetanilide': '[#6][#6]C(=O)Nc1ccccc1',
        'Primary_Secondary_Amine': '[NX3;H2,H1;!$(NC=O)]',
        'Alkyne': '[#6]=[#6]',
        'Benzoyl_Chloride': 'c1ccccc1C(=O)Cl',
        'Isothiocyanate': 'N=C=S',
        'Nitrile': '[C;H1,H2]#[N]',
        'Azide_Group': 'N=[N+]=[N-]',
        'Isocyanate': 'N=C=O',
        'Nitrobenzene': 'c1ccccc1N(=O)=O',
        'Alkene': 'C=C',
        'Phenol': '[O;H1]-c1ccccc1',
        'Enol': '[C;H2,H1]=[C;H2,H1]-[OH]',
        'Coumarin': 'c1ccc2c(c1)ccc(=O)o2'
    }

    alert_hits = 0
    for smarts in alert_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            alert_hits += 1
    total_alerts = len(alert_smarts)

    # === High Ionization Potential Substructure ===
    high_ip_smarts = '[#6][#6]S(=O)(=O)N'
    high_ip_atoms = mol.GetSubstructMatches(Chem.MolFromSmarts(high_ip_smarts))
    num_high_ip_atoms = len(high_ip_atoms)

    # === Final feature list ===
    return [
        molar_mass,
        logp_value,
        num_hb_acceptors,
        num_hb_donors,
        tpsa_value,
        num_rotatable_bonds,
        num_rings,
        num_aromatic_rings,
        fraction_sp3_carbons,
        f"{toxicophore_hits}/{total_toxicophores}",
        f"{alert_hits}/{total_alerts}",
        qed_value,
        lipinski_violations,
        molar_refractivity,
        num_high_ip_atoms
    ]

# === Read SMILES from input CSV ===
df = pd.read_csv(smiles_csv_path)
smiles_list = df['PUBCHEM_EXT_DATASOURCE_SMILES'].dropna().unique()

# === Write features to output CSV ===
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    header = [
        "SMILES",
        "Molecular_Weight",
        "LogP",
        "H_Acceptors",
        "H_Donors",
        "TPSA",
        "Rotable_Bonds",
        "Rings",
        "Aromatic_Rings",
        "SP3_Carbon",
        "Toxicophore_Presence",
        "Alerting_Structural_Motifs",
        "QED",
        "Lipinskis_Rule",
        "Molar_Refractivity",
        "Atoms_With_High_Ionization_Potential"
    ]
    writer.writerow(header)

    for smiles in smiles_list:
        features = extract_features(smiles)
        if features is None:
            writer.writerow([smiles] + ["INVALID_SMILES"] * (len(header) - 1))
        else:
            writer.writerow([smiles] + features)

print(f"✅ Feature extraction complete. Output saved to: {output_csv_path}")


In [ ]:


import csv
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, rdMolDescriptors

# === File paths ===
smiles_csv_path = r'C:\PhD\Project\Vero_76.csv'
output_csv_path = r'C:\PhD\Prompt_Engineering\Outputs2\DS5_Calc.csv'

# === Feature extraction function ===
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Basic molecular descriptors
    molar_mass = Descriptors.MolWt(mol)
    logp_value = Descriptors.MolLogP(mol)
    num_hb_acceptors = Lipinski.NumHAcceptors(mol)
    num_hb_donors = Lipinski.NumHDonors(mol)
    tpsa_value = rdMolDescriptors.CalcTPSA(mol)
    num_rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    fraction_sp3_carbons = Descriptors.FractionCSP3(mol)
    qed_value = QED.qed(mol)
    molar_refractivity = Descriptors.MolMR(mol)

    # Lipinski rule violations
    lipinski_violations = sum([
        molar_mass > 500,
        logp_value > 5,
        num_hb_acceptors > 10,
        num_hb_donors > 5
    ])

    # Ring info
    ring_info = mol.GetRingInfo()
    atom_rings = ring_info.AtomRings()
    num_rings = len(atom_rings)
    num_aromatic_rings = len([
        r for r in atom_rings if all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in r)
    ])

    # === Toxicophores ===
    toxicophore_smarts = {
        'Amide': '[NX3][CX3](=O)[#6]',
        'Sulfonamide': '[#6][#6]S(=O)(=O)N',
        'Carboxylic_Acid': '[CX3](=O)[OX2H1]',
        'Nitro_Aliphatic': '[#7]N=O',
        'Nitro_Aromatic': 'c1ccc([N+](=O)[O-])cc1',
        'Azide': '[NX3][NX2]=[NX2]',
        'Acyl_Chloride': '[CX3](=O)Cl',
        'Anhydride': '[CX3](=O)OC(=O)[#6]',
        'Hydroxamic_Acid': '[CX3](=O)N[OH]',
        'Beta_Diketone': '[C;!R](=O)[C;!R](=O)',
        'Peroxynitrite': '[O-][N+](=O)O',
        'General_Nitro': '[$([N+](=O)[O-])]'
    }

    toxicophore_hits = 0
    for smarts in toxicophore_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            toxicophore_hits += 1
    total_toxicophores = len(toxicophore_smarts)

    # === Alerting Structural Motifs ===
    alert_smarts = {
        'Aryl_Sulfonamide': '[#6][#6]S(=O)(=O)Nc1ccccc1',
        'Arylamide': 'CCc1ccc(cc1)C(=O)Nc2ccccc2',
        'Acetanilide': '[#6][#6]C(=O)Nc1ccccc1',
        'Primary_Secondary_Amine': '[NX3;H2,H1;!$(NC=O)]',
        'Alkyne': '[#6]=[#6]',
        'Benzoyl_Chloride': 'c1ccccc1C(=O)Cl',
        'Isothiocyanate': 'N=C=S',
        'Nitrile': '[C;H1,H2]#[N]',
        'Azide_Group': 'N=[N+]=[N-]',
        'Isocyanate': 'N=C=O',
        'Nitrobenzene': 'c1ccccc1N(=O)=O',
        'Alkene': 'C=C',
        'Phenol': '[O;H1]-c1ccccc1',
        'Enol': '[C;H2,H1]=[C;H2,H1]-[OH]',
        'Coumarin': 'c1ccc2c(c1)ccc(=O)o2'
    }

    alert_hits = 0
    for smarts in alert_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            alert_hits += 1
    total_alerts = len(alert_smarts)

    # === High Ionization Potential Substructure ===
    high_ip_smarts = '[#6][#6]S(=O)(=O)N'
    high_ip_atoms = mol.GetSubstructMatches(Chem.MolFromSmarts(high_ip_smarts))
    num_high_ip_atoms = len(high_ip_atoms)

    # === Final feature list ===
    return [
        molar_mass,
        logp_value,
        num_hb_acceptors,
        num_hb_donors,
        tpsa_value,
        num_rotatable_bonds,
        num_rings,
        num_aromatic_rings,
        fraction_sp3_carbons,
        f"{toxicophore_hits}/{total_toxicophores}",
        f"{alert_hits}/{total_alerts}",
        qed_value,
        lipinski_violations,
        molar_refractivity,
        num_high_ip_atoms
    ]

# === Read SMILES from input CSV ===
df = pd.read_csv(smiles_csv_path)
smiles_list = df['PUBCHEM_EXT_DATASOURCE_SMILES'].dropna().unique()

# === Write features to output CSV ===
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    header = [
        "SMILES",
        "Molecular_Weight",
        "LogP",
        "H_Acceptors",
        "H_Donors",
        "TPSA",
        "Rotable_Bonds",
        "Rings",
        "Aromatic_Rings",
        "SP3_Carbon",
        "Toxicophore_Presence",
        "Alerting_Structural_Motifs",
        "QED",
        "Lipinskis_Rule",
        "Molar_Refractivity",
        "Atoms_With_High_Ionization_Potential"
    ]
    writer.writerow(header)

    for smiles in smiles_list:
        features = extract_features(smiles)
        if features is None:
            writer.writerow([smiles] + ["INVALID_SMILES"] * (len(header) - 1))
        else:
            writer.writerow([smiles] + features)

print(f"✅ Feature extraction complete. Output saved to: {output_csv_path}")

In [ ]:

import csv
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, rdMolDescriptors

# === File paths ===
smiles_csv_path = r'C:\PhD\Project\THP-1.csv'
output_csv_path = r'C:\PhD\Prompt_Engineering\Outputs2\DS6_Calc.csv'

# === Feature extraction function ===
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Basic molecular descriptors
    molar_mass = Descriptors.MolWt(mol)
    logp_value = Descriptors.MolLogP(mol)
    num_hb_acceptors = Lipinski.NumHAcceptors(mol)
    num_hb_donors = Lipinski.NumHDonors(mol)
    tpsa_value = rdMolDescriptors.CalcTPSA(mol)
    num_rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    fraction_sp3_carbons = Descriptors.FractionCSP3(mol)
    qed_value = QED.qed(mol)
    molar_refractivity = Descriptors.MolMR(mol)

    # Lipinski rule violations
    lipinski_violations = sum([
        molar_mass > 500,
        logp_value > 5,
        num_hb_acceptors > 10,
        num_hb_donors > 5
    ])

    # Ring info
    ring_info = mol.GetRingInfo()
    atom_rings = ring_info.AtomRings()
    num_rings = len(atom_rings)
    num_aromatic_rings = len([
        r for r in atom_rings if all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in r)
    ])

    # === Toxicophores ===
    toxicophore_smarts = {
        'Amide': '[NX3][CX3](=O)[#6]',
        'Sulfonamide': '[#6][#6]S(=O)(=O)N',
        'Carboxylic_Acid': '[CX3](=O)[OX2H1]',
        'Nitro_Aliphatic': '[#7]N=O',
        'Nitro_Aromatic': 'c1ccc([N+](=O)[O-])cc1',
        'Azide': '[NX3][NX2]=[NX2]',
        'Acyl_Chloride': '[CX3](=O)Cl',
        'Anhydride': '[CX3](=O)OC(=O)[#6]',
        'Hydroxamic_Acid': '[CX3](=O)N[OH]',
        'Beta_Diketone': '[C;!R](=O)[C;!R](=O)',
        'Peroxynitrite': '[O-][N+](=O)O',
        'General_Nitro': '[$([N+](=O)[O-])]'
    }

    toxicophore_hits = 0
    for smarts in toxicophore_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            toxicophore_hits += 1
    total_toxicophores = len(toxicophore_smarts)

    # === Alerting Structural Motifs ===
    alert_smarts = {
        'Aryl_Sulfonamide': '[#6][#6]S(=O)(=O)Nc1ccccc1',
        'Arylamide': 'CCc1ccc(cc1)C(=O)Nc2ccccc2',
        'Acetanilide': '[#6][#6]C(=O)Nc1ccccc1',
        'Primary_Secondary_Amine': '[NX3;H2,H1;!$(NC=O)]',
        'Alkyne': '[#6]=[#6]',
        'Benzoyl_Chloride': 'c1ccccc1C(=O)Cl',
        'Isothiocyanate': 'N=C=S',
        'Nitrile': '[C;H1,H2]#[N]',
        'Azide_Group': 'N=[N+]=[N-]',
        'Isocyanate': 'N=C=O',
        'Nitrobenzene': 'c1ccccc1N(=O)=O',
        'Alkene': 'C=C',
        'Phenol': '[O;H1]-c1ccccc1',
        'Enol': '[C;H2,H1]=[C;H2,H1]-[OH]',
        'Coumarin': 'c1ccc2c(c1)ccc(=O)o2'
    }

    alert_hits = 0
    for smarts in alert_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            alert_hits += 1
    total_alerts = len(alert_smarts)

    # === High Ionization Potential Substructure ===
    high_ip_smarts = '[#6][#6]S(=O)(=O)N'
    high_ip_atoms = mol.GetSubstructMatches(Chem.MolFromSmarts(high_ip_smarts))
    num_high_ip_atoms = len(high_ip_atoms)

    # === Final feature list ===
    return [
        molar_mass,
        logp_value,
        num_hb_acceptors,
        num_hb_donors,
        tpsa_value,
        num_rotatable_bonds,
        num_rings,
        num_aromatic_rings,
        fraction_sp3_carbons,
        f"{toxicophore_hits}/{total_toxicophores}",
        f"{alert_hits}/{total_alerts}",
        qed_value,
        lipinski_violations,
        molar_refractivity,
        num_high_ip_atoms
    ]

# === Read SMILES from input CSV ===
df = pd.read_csv(smiles_csv_path)
smiles_list = df['PUBCHEM_EXT_DATASOURCE_SMILES'].dropna().unique()

# === Write features to output CSV ===
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    header = [
        "SMILES",
        "Molecular_Weight",
        "LogP",
        "H_Acceptors",
        "H_Donors",
        "TPSA",
        "Rotable_Bonds",
        "Rings",
        "Aromatic_Rings",
        "SP3_Carbon",
        "Toxicophore_Presence",
        "Alerting_Structural_Motifs",
        "QED",
        "Lipinskis_Rule",
        "Molar_Refractivity",
        "Atoms_With_High_Ionization_Potential"
    ]
    writer.writerow(header)

    for smiles in smiles_list:
        features = extract_features(smiles)
        if features is None:
            writer.writerow([smiles] + ["INVALID_SMILES"] * (len(header) - 1))
        else:
            writer.writerow([smiles] + features)

print(f"✅ Feature extraction complete. Output saved to: {output_csv_path}")


In [ ]:

import csv
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, rdMolDescriptors

# === File paths ===
smiles_csv_path = r'C:\PhD\Project\Vero_E6.csv'
output_csv_path = r'C:\PhD\Prompt_Engineering\Outputs2\DS7_Calc.csv'

# === Feature extraction function ===
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Basic molecular descriptors
    molar_mass = Descriptors.MolWt(mol)
    logp_value = Descriptors.MolLogP(mol)
    num_hb_acceptors = Lipinski.NumHAcceptors(mol)
    num_hb_donors = Lipinski.NumHDonors(mol)
    tpsa_value = rdMolDescriptors.CalcTPSA(mol)
    num_rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    fraction_sp3_carbons = Descriptors.FractionCSP3(mol)
    qed_value = QED.qed(mol)
    molar_refractivity = Descriptors.MolMR(mol)

    # Lipinski rule violations
    lipinski_violations = sum([
        molar_mass > 500,
        logp_value > 5,
        num_hb_acceptors > 10,
        num_hb_donors > 5
    ])

    # Ring info
    ring_info = mol.GetRingInfo()
    atom_rings = ring_info.AtomRings()
    num_rings = len(atom_rings)
    num_aromatic_rings = len([
        r for r in atom_rings if all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in r)
    ])

    # === Toxicophores ===
    toxicophore_smarts = {
        'Amide': '[NX3][CX3](=O)[#6]',
        'Sulfonamide': '[#6][#6]S(=O)(=O)N',
        'Carboxylic_Acid': '[CX3](=O)[OX2H1]',
        'Nitro_Aliphatic': '[#7]N=O',
        'Nitro_Aromatic': 'c1ccc([N+](=O)[O-])cc1',
        'Azide': '[NX3][NX2]=[NX2]',
        'Acyl_Chloride': '[CX3](=O)Cl',
        'Anhydride': '[CX3](=O)OC(=O)[#6]',
        'Hydroxamic_Acid': '[CX3](=O)N[OH]',
        'Beta_Diketone': '[C;!R](=O)[C;!R](=O)',
        'Peroxynitrite': '[O-][N+](=O)O',
        'General_Nitro': '[$([N+](=O)[O-])]'
    }

    toxicophore_hits = 0
    for smarts in toxicophore_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            toxicophore_hits += 1
    total_toxicophores = len(toxicophore_smarts)

    # === Alerting Structural Motifs ===
    alert_smarts = {
        'Aryl_Sulfonamide': '[#6][#6]S(=O)(=O)Nc1ccccc1',
        'Arylamide': 'CCc1ccc(cc1)C(=O)Nc2ccccc2',
        'Acetanilide': '[#6][#6]C(=O)Nc1ccccc1',
        'Primary_Secondary_Amine': '[NX3;H2,H1;!$(NC=O)]',
        'Alkyne': '[#6]=[#6]',
        'Benzoyl_Chloride': 'c1ccccc1C(=O)Cl',
        'Isothiocyanate': 'N=C=S',
        'Nitrile': '[C;H1,H2]#[N]',
        'Azide_Group': 'N=[N+]=[N-]',
        'Isocyanate': 'N=C=O',
        'Nitrobenzene': 'c1ccccc1N(=O)=O',
        'Alkene': 'C=C',
        'Phenol': '[O;H1]-c1ccccc1',
        'Enol': '[C;H2,H1]=[C;H2,H1]-[OH]',
        'Coumarin': 'c1ccc2c(c1)ccc(=O)o2'
    }

    alert_hits = 0
    for smarts in alert_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            alert_hits += 1
    total_alerts = len(alert_smarts)

    # === High Ionization Potential Substructure ===
    high_ip_smarts = '[#6][#6]S(=O)(=O)N'
    high_ip_atoms = mol.GetSubstructMatches(Chem.MolFromSmarts(high_ip_smarts))
    num_high_ip_atoms = len(high_ip_atoms)

    # === Final feature list ===
    return [
        molar_mass,
        logp_value,
        num_hb_acceptors,
        num_hb_donors,
        tpsa_value,
        num_rotatable_bonds,
        num_rings,
        num_aromatic_rings,
        fraction_sp3_carbons,
        f"{toxicophore_hits}/{total_toxicophores}",
        f"{alert_hits}/{total_alerts}",
        qed_value,
        lipinski_violations,
        molar_refractivity,
        num_high_ip_atoms
    ]

# === Read SMILES from input CSV ===
df = pd.read_csv(smiles_csv_path)
smiles_list = df['PUBCHEM_EXT_DATASOURCE_SMILES'].dropna().unique()

# === Write features to output CSV ===
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    header = [
        "SMILES",
        "Molecular_Weight",
        "LogP",
        "H_Acceptors",
        "H_Donors",
        "TPSA",
        "Rotable_Bonds",
        "Rings",
        "Aromatic_Rings",
        "SP3_Carbon",
        "Toxicophore_Presence",
        "Alerting_Structural_Motifs",
        "QED",
        "Lipinskis_Rule",
        "Molar_Refractivity",
        "Atoms_With_High_Ionization_Potential"
    ]
    writer.writerow(header)

    for smiles in smiles_list:
        features = extract_features(smiles)
        if features is None:
            writer.writerow([smiles] + ["INVALID_SMILES"] * (len(header) - 1))
        else:
            writer.writerow([smiles] + features)

print(f"✅ Feature extraction complete. Output saved to: {output_csv_path}")


In [ ]:
import csv
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, rdMolDescriptors

# === File paths ===
smiles_csv_path = r'C:\PhD\Project\SA9-PAX8.csv'
output_csv_path = r'C:\PhD\Prompt_Engineering\Outputs2\DS8_Calc.csv'

# === Feature extraction function ===
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Basic molecular descriptors
    molar_mass = Descriptors.MolWt(mol)
    logp_value = Descriptors.MolLogP(mol)
    num_hb_acceptors = Lipinski.NumHAcceptors(mol)
    num_hb_donors = Lipinski.NumHDonors(mol)
    tpsa_value = rdMolDescriptors.CalcTPSA(mol)
    num_rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    fraction_sp3_carbons = Descriptors.FractionCSP3(mol)
    qed_value = QED.qed(mol)
    molar_refractivity = Descriptors.MolMR(mol)

    # Lipinski rule violations
    lipinski_violations = sum([
        molar_mass > 500,
        logp_value > 5,
        num_hb_acceptors > 10,
        num_hb_donors > 5
    ])

    # Ring info
    ring_info = mol.GetRingInfo()
    atom_rings = ring_info.AtomRings()
    num_rings = len(atom_rings)
    num_aromatic_rings = len([
        r for r in atom_rings if all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in r)
    ])

    # === Toxicophores ===
    toxicophore_smarts = {
        'Amide': '[NX3][CX3](=O)[#6]',
        'Sulfonamide': '[#6][#6]S(=O)(=O)N',
        'Carboxylic_Acid': '[CX3](=O)[OX2H1]',
        'Nitro_Aliphatic': '[#7]N=O',
        'Nitro_Aromatic': 'c1ccc([N+](=O)[O-])cc1',
        'Azide': '[NX3][NX2]=[NX2]',
        'Acyl_Chloride': '[CX3](=O)Cl',
        'Anhydride': '[CX3](=O)OC(=O)[#6]',
        'Hydroxamic_Acid': '[CX3](=O)N[OH]',
        'Beta_Diketone': '[C;!R](=O)[C;!R](=O)',
        'Peroxynitrite': '[O-][N+](=O)O',
        'General_Nitro': '[$([N+](=O)[O-])]'
    }

    toxicophore_hits = 0
    for smarts in toxicophore_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            toxicophore_hits += 1
    total_toxicophores = len(toxicophore_smarts)

    # === Alerting Structural Motifs ===
    alert_smarts = {
        'Aryl_Sulfonamide': '[#6][#6]S(=O)(=O)Nc1ccccc1',
        'Arylamide': 'CCc1ccc(cc1)C(=O)Nc2ccccc2',
        'Acetanilide': '[#6][#6]C(=O)Nc1ccccc1',
        'Primary_Secondary_Amine': '[NX3;H2,H1;!$(NC=O)]',
        'Alkyne': '[#6]=[#6]',
        'Benzoyl_Chloride': 'c1ccccc1C(=O)Cl',
        'Isothiocyanate': 'N=C=S',
        'Nitrile': '[C;H1,H2]#[N]',
        'Azide_Group': 'N=[N+]=[N-]',
        'Isocyanate': 'N=C=O',
        'Nitrobenzene': 'c1ccccc1N(=O)=O',
        'Alkene': 'C=C',
        'Phenol': '[O;H1]-c1ccccc1',
        'Enol': '[C;H2,H1]=[C;H2,H1]-[OH]',
        'Coumarin': 'c1ccc2c(c1)ccc(=O)o2'
    }

    alert_hits = 0
    for smarts in alert_smarts.values():
        pattern = Chem.MolFromSmarts(smarts)
        if mol.HasSubstructMatch(pattern):
            alert_hits += 1
    total_alerts = len(alert_smarts)

    # === High Ionization Potential Substructure ===
    high_ip_smarts = '[#6][#6]S(=O)(=O)N'
    high_ip_atoms = mol.GetSubstructMatches(Chem.MolFromSmarts(high_ip_smarts))
    num_high_ip_atoms = len(high_ip_atoms)

    # === Final feature list ===
    return [
        molar_mass,
        logp_value,
        num_hb_acceptors,
        num_hb_donors,
        tpsa_value,
        num_rotatable_bonds,
        num_rings,
        num_aromatic_rings,
        fraction_sp3_carbons,
        f"{toxicophore_hits}/{total_toxicophores}",
        f"{alert_hits}/{total_alerts}",
        qed_value,
        lipinski_violations,
        molar_refractivity,
        num_high_ip_atoms
    ]

# === Read SMILES from input CSV ===
df = pd.read_csv(smiles_csv_path)
smiles_list = df['PUBCHEM_EXT_DATASOURCE_SMILES'].dropna().unique()

# === Write features to output CSV ===
with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    header = [
        "SMILES",
        "Molecular_Weight",
        "LogP",
        "H_Acceptors",
        "H_Donors",
        "TPSA",
        "Rotable_Bonds",
        "Rings",
        "Aromatic_Rings",
        "SP3_Carbon",
        "Toxicophore_Presence",
        "Alerting_Structural_Motifs",
        "QED",
        "Lipinskis_Rule",
        "Molar_Refractivity",
        "Atoms_With_High_Ionization_Potential"
    ]
    writer.writerow(header)

    for smiles in smiles_list:
        features = extract_features(smiles)
        if features is None:
            writer.writerow([smiles] + ["INVALID_SMILES"] * (len(header) - 1))
        else:
            writer.writerow([smiles] + features)

print(f"✅ Feature extraction complete. Output saved to: {output_csv_path}")
